In [1]:
# %%
import os

os.environ["TOKENIZERS_PARALLELISM"] = 'false'
os.environ["WANDB_DIR"] = '/projets/melodi/gsantoss/wandbt'

from cmatcher.owl_utils import *
from cmatcher.cqa_search import *
from cmatcher.eval_utils import *
from transformers import AutoTokenizer
import dill
import torch
import torch.optim as optim
import copy
import tqdm
from cmatcher.model import *
from tqdm.auto import tqdm
import random
import wandb

import argparse

torch.manual_seed(0)
random.seed(0)

In [2]:
# %%


# %%


def parse_arguments():
    arg_parser = argparse.ArgumentParser(description='')

    arg_parser.add_argument('--sweep', dest='sweep', nargs='?', type=int)

    return arg_parser.parse_args()


# %%


# args = parse_arguments()

test_onts = ['cmt', 'conference', 'confOf', 'edas', 'ekaw']
language_models = ['BAAI/bge-base-en', 'infgrad/stella-base-en-v2', 'BAAI/bge-large-en-v1.5', 'llmrails/ember-v1',
                   'thenlper/gte-large']
architectures = ['lm', 'gnn', 'sgnn']
lm_grad = ['none', 'grad']
pred = ['none', 'pred']
dephs = [1, 2, 3, 4]


def all_combinations():
    combs = []
    for to in test_onts:
        for lm in language_models:
            for a in architectures:
                if a == 'lm':
                    combs.append((to, lm, a, 'grad', 'none', 0))
                    continue
                for g in lm_grad:
                    for p in pred:
                        for d in dephs:
                            combs.append((to, lm, a, g, p, d))

    return combs

In [3]:
# test_ont, language_model, architecture, grad, cpred, depth = all_combinations()[args.sweep]
# # test_ont, language_model, architecture, grad, cpred, depth = all_combinations()[0]
# 
# config = {
#     'test_ont': test_ont,
#     'learning_rate': 0.00001,
#     'language_model': language_model,
#     'architecture': architecture,
#     'pred': cpred,
#     'epochs': 5,
#     'batch_size': 2,
#     'evm_th': 0.9,
#     'ev_sim_threshold': 0.8,
#     'sim_margin': 0.8,
#     'depth': depth,
#     'grad': grad
# }


In [4]:
config = {
    'test_ont': 'confOf',
    'learning_rate': 0.00001,
    'language_model': 'BAAI/bge-base-en',
    'architecture': 'lm',
    'pred': 'none',
    'epochs': 5,
    'batch_size': 2,
    'evm_th': 0.9,
    'ev_sim_threshold': 0.8,
    'sim_margin': 0.8,
    'depth': 0,
    'grad': 'grad'
}

In [5]:







ontology_paths = {
    'edas.owl': '/projets/melodi/gsantoss/data/oaei/tracks/conference/onts/edas.owl',
    'ekaw.owl': '/projets/melodi/gsantoss/data/oaei/tracks/conference/onts/ekaw.owl',
    'confOf.owl': '/projets/melodi/gsantoss/data/oaei/tracks/conference/onts/confOf.owl',
    'conference.owl': '/projets/melodi/gsantoss/data/oaei/tracks/conference/onts/conference.owl',
    'cmt.owl': '/projets/melodi/gsantoss/data/oaei/tracks/conference/onts/cmt.owl',
}

cqa_path = '/projets/melodi/gsantoss/data/complex/CQAs'
entities_path = '/projets/melodi/gsantoss/data/complex/entities-cqas'

if os.path.exists('/projets/melodi/gsantoss/tmp/idata.pkl'):
    with open('/projets/melodi/gsantoss/tmp/idata.pkl', 'rb') as f:
        train_ont_cqa_subg = dill.load(f)
        print('loaded from cache.')
else:
    with open('/projets/melodi/gsantoss/tmp/idata.pkl', 'wb') as f:
        dill.dump(load_entities(entities_path, ontology_paths), f)

isg = load_sg(entities_path, ontology_paths)

cqas = load_cqas(cqa_path)
raw_data = build_raw_data(train_ont_cqa_subg, cqas)

test_ont = config['test_ont']

if os.path.exists(f'/projets/melodi/gsantoss/tmp/{test_ont}.pkl'):
    with open(f'/projets/melodi/gsantoss/tmp/{test_ont}.pkl', 'rb') as f:
        ifd, mc, mp, fres = dill.load(f)
        print('loaded from cache.')
else:
    ifd, mc, mp, fres = build_raw_ts(f'/projets/melodi/gsantoss/data/oaei/tracks/conference/onts/{test_ont}.owl',
                                     isg[test_ont],
                                     workers=4)
    with open(f'/projets/melodi/gsantoss/tmp/{test_ont}.pkl', 'wb') as f:
        dill.dump((ifd, mc, mp, fres), f)

conts_cqa_subg = copy.deepcopy(train_ont_cqa_subg)
del conts_cqa_subg[test_ont]

tokenizer = AutoTokenizer.from_pretrained(config['language_model'])

root_entities, graph_data, cq, cqid, caq, cqmask, tor = prepare_eval_dataset(test_ont, cqas, ifd, tokenizer, mc, mp,
                                                                             fres, flat_bn=False)

wandb.init(
    project='tcmatcher',
    config=config,
    # group=f'{language_model}-{architecture}-{cpred}-{grad}',
    settings=wandb.Settings(_disable_stats=True, _disable_meta=True)
)

print(config)

# %%
print('start training')

# %%
model = Model(config['language_model'], d=config['depth'], lm_grad=config['grad'] == 'grad')
model.cuda(0)


# %%


def evm(model, dataset, th=0.5):
    model.eval()

    res = []
    print('begin evm')
    for batch in DataLoader(dataset, batch_size=2):
        with torch.no_grad():
            cqs, sbgs, _ = model(cqa=batch.cqs.cuda(0), positive_sbg=(batch.x_sf.cuda(0), batch.x_s.cuda(0),
                                                                      batch.edge_index_s.cuda(0),
                                                                      batch.edge_feat_sf.cuda(0),
                                                                      batch.edge_feat_s.cuda(0)))

            isbgs = sbgs[batch.rsi]

            sim = torch.cosine_similarity(cqs, isbgs) > th
            res.append(sim)

    res = torch.cat(res, dim=0)
    print('end evm')
    return (res.sum() / res.size(0)).item()


# model = Model(config['language_model'], d=config['depth'], lm_grad=config['grad'] == 'grad')

# %%
optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'])

lh = []
evh = []
epochs = config['epochs']
batch_size = config['batch_size']
progress = None

triplet_loss = nn.TripletMarginWithDistanceLoss(
    distance_function=lambda x, y: 1.0 - torch.cosine_similarity(x, y), margin=config['sim_margin'])

print('build datasets')
dataset = CQADataset(tokenizer, conts_cqa_subg, raw_data[test_ont], filter_bn=False, flat_bn=False)
loader = DataLoader(dataset, batch_size=batch_size)

cqloader = DataLoader(cqid, batch_size=batch_size, shuffle=False)
acqloader = [DataLoader(a, batch_size=batch_size, shuffle=False) for a in caq]
graph_loader = DataLoader(graph_data, batch_size=batch_size, shuffle=False)
# %%



loaded from cache.


  0%|          | 0/101 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

loaded from cache.


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: guihss-cs (ghss). Use `wandb login --relogin` to force relogin


{'test_ont': 'confOf', 'learning_rate': 1e-05, 'language_model': 'BAAI/bge-base-en', 'architecture': 'lm', 'pred': 'none', 'epochs': 5, 'batch_size': 2, 'evm_th': 0.9, 'ev_sim_threshold': 0.8, 'sim_margin': 0.8, 'depth': 0, 'grad': 'grad'}
start training
build datasets


ValueError: text input must of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

In [ ]:


print('data prepared')
model.find_unused_parameters = False
if not progress:
    progress = tqdm(total=epochs * len(loader))

print('start training')
evh.append(evm(model, dataset, th=config["ev_sim_threshold"]))
eval_test(model, cqloader, graph_loader, cq, root_entities, fres, acqloader, cqmask, tor)
wandb.log({'global/acc': evh[-1]})

for e in range(epochs):

    model.train()

    el = []
    for batch in loader:
        optimizer.zero_grad()

        cqs, sbgs, nsbg = model(cqa=batch.cqs.cuda(0), positive_sbg=(batch.x_sf.cuda(0), batch.x_s.cuda(0),
                                                                     batch.edge_index_s.cuda(0),
                                                                     batch.edge_feat_sf.cuda(0),
                                                                     batch.edge_feat_s.cuda(0)),
                                negative_sbg=(batch.x_nf.cuda(0), batch.x_n.cuda(0),
                                              batch.edge_index_n.cuda(0), batch.edge_feat_nf.cuda(0),
                                              batch.edge_feat_n.cuda(0)))

        isbgs = sbgs[batch.rsi]
        isbgn = nsbg[batch.rni]

        loss = triplet_loss(cqs, isbgs, isbgn)
        el.append(loss.detach())
        loss.backward()

        optimizer.step()
        progress.update(1)

    lh.append(torch.stack(el).mean().item())

    evh.append(evm(model, dataset, th=config["ev_sim_threshold"]))
    eval_test(model, cqloader, graph_loader, cq, root_entities, fres, acqloader, cqmask, tor)
    wandb.log({'global/acc': evh[-1], 'global/loss': lh[-1]})

progress.close()

wandb.finish()

torch.save(model.state_dict(), f'/projets/melodi/gsantoss/models/cmatcher.pt')
# %%

In [6]:
model = Model(config['language_model'], d=config['depth'], lm_grad=config['grad'] == 'grad')
model.load_state_dict(torch.load(f'/projets/melodi/gsantoss/models/cmatcher.pt'))
model.cuda(0)
model.eval()

Model(
  (emb1): BertEmb(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
                (Lay

In [7]:
dataset = CQADataset(tokenizer, conts_cqa_subg, raw_data[test_ont], filter_bn=False)
evm(model, dataset, th=config["ev_sim_threshold"])

begin evm
end evm


0.9438202381134033

In [11]:
def eval_test(model, cqloader, graph_loader, cq, root_entities, res, caq, cqmask, tor):
    model.eval()

    cqeb = embed_cqas(model, cqloader)
    aembs = [embed_cqas(model, a) for a in caq]

    graph_embeddings = embed_subg(model, graph_loader)

    avgps = []
    rcs = []
    fms = []

    for t in torch.arange(0, 1, 0.05):
        avgp, rc, fm = get_apr(eval_metrics(cq, cqeb, graph_embeddings, root_entities, res, th=t))
        avgps.append(avgp)
        rcs.append(rc)
        fms.append(fm)

    bv = torch.tensor(fms).argmax().item()

    print({'global/bt': bv * 0.05, 'global/avgp': avgps[bv], 'global/rec': rcs[bv], 'global/afm': fms[bv]})

    gavgps = 0
    grcs = 0
    gfms = 0

    for i in range(len(tor)):
        avgps = []
        rcs = []
        fms = []
        for t in torch.arange(0, 1, 0.05):
            metrics = eval_metrics(cq, aembs[i], graph_embeddings, root_entities, res, th=t,
                                   cqm=[x[i] for x in cqmask])
            avgp, rc, fm = get_apr(metrics)
            avgps.append(avgp)
            rcs.append(rc)
            fms.append(fm)

        bv = torch.tensor(fms).argmax().item()

        gavgps += avgps[bv]
        grcs += rcs[bv]
        gfms += fms[bv]

    gavgps /= len(tor)
    grcs /= len(tor)
    gfms /= len(tor)

    print({'global/gavgp': gavgps, 'global/grec': grcs, 'global/gafm': gfms})
    
    
    
eval_test(model, cqloader, graph_loader, cq, root_entities, fres, acqloader, cqmask, tor)


{'global/bt': 0.8, 'global/avgp': 1.0, 'global/rec': 1.0, 'global/afm': 1.0}
{'global/gavgp': 0.6888581190051778, 'global/grec': 0.6382575757575757, 'global/gafm': 0.6600765654270265}
